# Transform Customer Data

## Revomove records with Null Customer_id

In [0]:
SELECT *FROM gizmobox.bronze.v_customers WHERE customer_id is not null;

## Remove exact duplicate records

In [0]:
SELECT DISTINCT 
*FROM gizmobox.bronze.v_customers 
WHERE customer_id is not null 
order by customer_id;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_customers_distinct
AS
SELECT DISTINCT 
*FROM gizmobox.bronze.v_customers 
WHERE customer_id is not null 
order by customer_id;

## Remove duplicate records based on created_timestamp

Esta idea no es muy recomendable para eliminar duplicados , con columnas diferentes

In [0]:
SELECT DISTINCT 
customer_id,
max(created_timestamp),
max(customer_name),
max(date_of_birth),
max(email),
max(member_since),
max(telephone)
FROM gizmobox.bronze.v_customers 
WHERE customer_id is not null  
group by customer_id
order by customer_id;

Esta es la idea mas recomendabl utilizar

In [0]:
SELECT customer_id,
MAX(created_timestamp) AS max_created_timestamp
FROM tv_customers_distinct
group by customer_id;

Lo que hacemo es utilizar  una expresion de tabla comun par aunir esta tabla con la vista creada anteormente

In [0]:
CREATE OR REPLACE TEMPORARY VIEW tv_max
AS
SELECT customer_id,
MAX(created_timestamp) AS max_created_timestamp
FROM tv_customers_distinct
group by customer_id

In [0]:
SELECT c.* FROM tv_max  m
join tv_customers_distinct c
on m.customer_id = c.customer_id and m.max_created_timestamp = c.created_timestamp

## Cast the column values to the correct data type

In [0]:
SELECT 
  CAST(c.created_timestamp AS TIMESTAMP) as created_timestamp,
  c.customer_id,
  c.customer_name,
  CAST(c.date_of_birth as DATE) as date_of_birth,
  c.email,
  CAST(c.member_since as DATE) as member_since,
  c.telephone
FROM tv_max  m
join tv_customers_distinct c
on m.customer_id = c.customer_id and m.max_created_timestamp = c.created_timestamp

## Write Data to en Delta Table

Esta tabla no es externa porque simplemente porque no tiene la ubicacion o location por la cual resulta ser una talba gestionada o managed, y sabemos que todas las tablas managed son tablas delta (ACID,TRAVEL TIME, ENFORMCED SCHEMA)

In [0]:
CREATE TABLE gizmobox.silver.customers
AS
  SELECT 
      CAST(c.created_timestamp AS TIMESTAMP) as created_timestamp,
      c.customer_id,
      c.customer_name,
      CAST(c.date_of_birth as DATE) as date_of_birth,
      c.email,
      CAST(c.member_since as DATE) as member_since,
      c.telephone
  FROM tv_max  m
        join tv_customers_distinct c
        on 
          m.customer_id = c.customer_id and 
          m.max_created_timestamp = c.created_timestamp

In [0]:
DESC EXTENDED gizmobox.silver.customers;